In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import sklearn

In [8]:
import requests
def telegram_bot_sendtext(bot_message):
    
    bot_token = '661544421:AAHuZSMukZmFlhU-npE23CowogP375a0au4'
    bot_chatID = '-225157366'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()


In [9]:
train_y = (pd.read_csv("./data/train.csv", encoding='big5')['Y1']=="Y")+0

train_x = pd.read_csv("./data/0915_train_pro.csv").drop(columns = 'Unnamed: 0')

test_x = pd.read_csv("./data/0915_test_pro.csv").drop(columns = 'Unnamed: 0')

/home/fio/anaconda3/envs/gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
from imblearn.under_sampling import RandomUnderSampler

RS = RandomUnderSampler(random_state=8787)

train_x, train_y = RS.fit_resample(train_x, train_y)
train_x = pd.DataFrame(train_x)
train_x.columns = test_x.columns
train_x.shape

(4000, 294)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.4, random_state = 8787)
#x_train, y_train = train_x, train_y

In [12]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=100, tree_method='gpu_hist', slient= 0, max_depth=4)

model.fit(x_train, y_train)
print("Score is", sklearn.metrics.accuracy_score(model.predict(x_valid), y_valid))

Score is 0.736875


In [13]:
models = []
models.append(model)

In [14]:
tests = model.predict(test_x).reshape(-1,1)
valids = model.predict(x_valid).reshape(-1, 1)

In [15]:
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from keras import regularizers
from keras.initializers import he_normal
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

from keras.callbacks import TensorBoard
import time
from time import time as currentTime
import tensorflow as tf
import keras

In [16]:
def build_model():
    in_out = keras.Input(shape = (294,), name = "input")
    out=in_out
    for i in [100,100, 100]:
        out1 = Dense(i, activation = "relu")(out)
        out2 = Dense(i, activation = "sigmoid")(out)
        out = keras.layers.concatenate([out1, out2])
        out = Dropout(0.3)(out)
    output = Dense(2, activation = "softmax")(out)

    model = keras.Model(
        inputs = [in_out],
        outputs = output
    )
    model.compile(loss = "categorical_crossentropy",
              optimizer = 'Adam', metrics = ["acc"])
    return model

In [17]:
from keras.callbacks import TensorBoard
import time
from time import time as currentTime
import tensorflow as tf
tensorboard = TensorBoard(log_dir="logs/{}".format(currentTime()))

In [18]:
y_train_util = keras.utils.np_utils.to_categorical(y_train)
y_valid_util = keras.utils.np_utils.to_categorical(y_valid)

In [19]:
model = build_model()
model.fit(x_train, y_train_util, epochs=18,
    verbose=0, validation_data = [x_valid, y_valid_util], validation_freq=3,
          callbacks=[tensorboard])
print("Model %d score %.2f"%(1, model.evaluate(x_valid, y_valid_util)[1]))

1600/1600 [==============================] - 0s 17us/step
Model 1 score 0.71


In [20]:
for i in range(80):
    model = build_model()
    model.fit(x_train, y_train_util, epochs=18,
        verbose=0, validation_data = [x_valid, y_valid_util], validation_freq=3,
              callbacks=[tensorboard])
    print("Model %d score %.2f \n"%(i, model.evaluate(x_valid, y_valid_util)[1]))
    valids = np.concatenate((valids, model.predict(x_valid).argmax(axis=1).reshape(-1,1)),
                            axis = 1)
    tests = np.concatenate((tests, model.predict(test_x).argmax(axis=1).reshape(-1,1)),
                            axis = 1)
    models.append(model)

1600/1600 [==============================] - 0s 17us/step
Model 0 score 0.72 

1600/1600 [==============================] - 0s 18us/step
Model 1 score 0.72 

1600/1600 [==============================] - 0s 19us/step
Model 2 score 0.70 

1600/1600 [==============================] - 0s 19us/step
Model 3 score 0.72 

1600/1600 [==============================] - 0s 20us/step
Model 4 score 0.72 

1600/1600 [==============================] - 0s 20us/step
Model 5 score 0.71 

1600/1600 [==============================] - 0s 28us/step
Model 19 score 0.72 

1600/1600 [==============================] - 0s 28us/step
Model 20 score 0.69 

1600/1600 [==============================] - 0s 29us/step
Model 21 score 0.72 

1600/1600 [==============================] - 0s 31us/step
Model 22 score 0.71 

1600/1600 [==============================] - 0s 30us/step
Model 23 score 0.72 

1600/1600 [==============================] - 0s 31us/step
Model 24 score 0.72 

1600/1600 [==============================] - 0

In [22]:
"finsih"

'finsih'

In [25]:
temp = valids

In [31]:
temp[0].shape

(81,)

In [50]:
temp_valid = []
for i in range(temp.shape[0]):    
    counts = np.bincount(temp[i])
    #返回众数
    temp_valid.append(np.argmax(counts))
temp_valid = np.array(temp_valid)

In [49]:
from sklearn.metrics import accuracy_score
accuracy_score(np.array(temp_valid), y_valid)

0.73375

81

In [172]:
ready_out_la = keras.Input(shape = (valids.shape[1],), name = "input")
ready_out=ready_out_la
for i in [1300, 300]:
    ready_out1 = Dense(i, activation = "relu")(ready_out)
    ready_out2 = Dense(i, activation = "sigmoid")(ready_out)
    ready_out = keras.layers.concatenate([ready_out1, ready_out2])
    ready_out = Dropout(0.3)(ready_out)
ready_output = Dense(2, activation = "softmax")(ready_out)

model = keras.Model(
    inputs = [ready_out_la],
    outputs = ready_output
)
model.compile(loss = "categorical_crossentropy",
          optimizer = 'Adam', metrics = ["acc"])
model.summary()

Model: "model_106"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 81)           0                                            
__________________________________________________________________________________________________
dense_733 (Dense)               (None, 1300)         106600      input[0][0]                      
__________________________________________________________________________________________________
dense_734 (Dense)               (None, 1300)         106600      input[0][0]                      
__________________________________________________________________________________________________
concatenate_314 (Concatenate)   (None, 2600)         0           dense_733[0][0]                  
                                                                 dense_734[0][0]          

In [173]:
train_valid_x, test_valid_x, train_valid_y, test_valid_y = train_test_split(temp, y_valid_util, test_size = 0.1, random_state = 8787)

In [176]:
model.fit(train_valid_x, train_valid_y, epochs=85, verbose=1,
          validation_data= [test_valid_x, test_valid_y], validation_freq= 3)

Train on 1440 samples, validate on 160 samples
Epoch 1/85
1440/1440 [==============================] - 0s 130us/step - loss: 0.3180 - acc: 0.8653
Epoch 2/85
1440/1440 [==============================] - 0s 124us/step - loss: 0.3019 - acc: 0.8771
Epoch 3/85
1440/1440 [==============================] - 0s 134us/step - loss: 0.3378 - acc: 0.8694 - val_loss: 0.9839 - val_acc: 0.6937
Epoch 4/85
1440/1440 [==============================] - 0s 123us/step - loss: 0.3181 - acc: 0.8750
Epoch 5/85
1440/1440 [==============================] - 0s 125us/step - loss: 0.3151 - acc: 0.8715
Epoch 6/85
1440/1440 [==============================] - 0s 134us/step - loss: 0.3118 - acc: 0.8757 - val_loss: 1.0549 - val_acc: 0.7250
Epoch 7/85
1440/1440 [==============================] - 0s 126us/step - loss: 0.3069 - acc: 0.8729
Epoch 8/85
1440/1440 [==============================] - 0s 126us/step - loss: 0.3094 - acc: 0.8826
Epoch 9/85
1440/1440 [==============================] - 0s 136us/step - loss: 0.3133 - 

1440/1440 [==============================] - 0s 126us/step - loss: 0.2972 - acc: 0.8882
Epoch 74/85
1440/1440 [==============================] - 0s 125us/step - loss: 0.2897 - acc: 0.8882
Epoch 75/85
1440/1440 [==============================] - 0s 132us/step - loss: 0.2989 - acc: 0.8868 - val_loss: 1.1208 - val_acc: 0.7063
Epoch 76/85
1440/1440 [==============================] - 0s 124us/step - loss: 0.2943 - acc: 0.8861
Epoch 77/85
1440/1440 [==============================] - 0s 126us/step - loss: 0.2896 - acc: 0.8861
Epoch 78/85
1440/1440 [==============================] - 0s 132us/step - loss: 0.2793 - acc: 0.8875 - val_loss: 1.1395 - val_acc: 0.6875
Epoch 79/85
1440/1440 [==============================] - 0s 124us/step - loss: 0.2878 - acc: 0.8910
Epoch 80/85
1440/1440 [==============================] - 0s 123us/step - loss: 0.2852 - acc: 0.8903
Epoch 81/85
1440/1440 [==============================] - 0s 132us/step - loss: 0.2797 - acc: 0.8903 - val_loss: 1.1562 - val_acc: 0.7063
E

In [177]:
model.evaluate(train_valid_x, train_valid_y), model.evaluate(test_valid_x, test_valid_y)

160/160 [==============================] - 0s 75us/step


([0.2672401196426816, 0.8993055555555556], [1.16314594745636, 0.70625])

In [159]:
model.evaluate(train_valid_x, train_valid_y), model.evaluate(test_valid_x, test_valid_y)

160/160 [==============================] - 0s 73us/step


([0.2926975283357832, 0.88125], [1.0258247077465057, 0.7])

In [155]:
model.evaluate(train_valid_x, train_valid_y), model.evaluate(test_valid_x, test_valid_y)

160/160 [==============================] - 0s 72us/step


([0.29432350595792134, 0.8833333333333333], [1.096831452846527, 0.725])

In [220]:
train_valid_x, test_valid_x, train_valid_y, test_valid_y = train_test_split(temp, y_valid, test_size = 0.1, random_state = 8787)
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=100, tree_method='gpu_hist', slient= 0, max_depth=10)

model.fit(train_valid_x, train_valid_y)
print("Score is", sklearn.metrics.accuracy_score(model.predict(test_valid_x), test_valid_y))

Score is 0.7125


In [221]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
rf.fit(train_valid_x, train_valid_y)
print("Score is", sklearn.metrics.accuracy_score(rf.predict(test_valid_x), test_valid_y))

Score is 0.70625


結果xgboost最好，所以我使用xgboost來投票

In [223]:
#train_valid_x, test_valid_x, train_valid_y, test_valid_y = train_test_split(temp, y_valid, test_size = 0.1, random_state = 8787)
train_valid_x, train_valid_y = temp, y_valid

import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=100, tree_method='gpu_hist', slient= 0, max_depth=10)

model.fit(train_valid_x, train_valid_y)
print("Score is", sklearn.metrics.accuracy_score(model.predict(train_valid_x), train_valid_y))

Score is 0.899375


In [224]:
tests.shape

(150000, 81)

In [227]:
result = model.predict(tests)

In [231]:
submit = pd.read_csv("./data/submit_test.csv")

submit.Ypred = result

submit.to_csv("./data/submit/0915_night.csv")